In [1]:
import requests
import re
from bs4 import BeautifulSoup as BS
from IPython.core.display import HTML
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Read in Data

In [2]:
movie = pd.read_csv("../Data/IMDb_Movies.csv")

In [3]:
score = pd.read_csv("../Data/IMDb_Score.csv")

In [4]:
rt = pd.read_csv("../Data/Rotten_Tomatoes.csv")

### Movie and RT combined

In [5]:
m_rt = movie.merge(rt, how = 'outer', on = 'Title_ID')

### Movie and Score Combined

In [6]:
m_s = movie.merge(score, how = 'outer', on = 'Score_ID')

### Movie, Score, and RT in One DataFrame

In [7]:
all_df = m_rt.merge(score, how = "left", on = "Score_ID")

# Further Analysis

## Question 5: What is the average length of the movies?

#### The longest movie?

#### The shortest movie?

## Question 6: What is the average movie rating?

## Question 7: What percentage of movies were "approved" with the Hayes Coded compared to newer movies with the MPAA modern rating system?

## Question 8: What is the range of years of the movies in the Top 250 List?

#### Are there outliers?

#### What decade were most films released in?

## Question 9: Which directors appear more than once?

## Question 10: What is the average difference between critics' scores and audiences' scores on Rotten Tomotatoes?

## Question 11: How do US-reviewers compare to non-US reviewers on IMDb?

#### Who tends to write more reviews?

#### What is the average scores between the two?

#### What genre interests US-reviewers more? What about non-US reviewers?

NOTE: important to note that the term US and non-US is based on the country the reviewer left the review...does not necessarily indicate that reviewers' citizenship

## Question 12: How many movies are a part of the same cinematic "universe"?

## Question 13: Do the movies on the Top 250 List tend to be nominated for several awards (i.e., more than 5)?

## Question 14: Which distributor has been a part of the most movies in the Top 250 Movie List?

### Random Code for Reference

code for font colors

In [8]:
### <font color='royalblue'> Visual (Question 2) </font>

see all columns

In [9]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
all_df

,Title_ID,Title,Director,Rating,Genre,Runtime,Release_Year,Award_Noms,Award_Wins,Score_ID,Tomatometer,RT_Audience_Score,RT_Critic_Reviews,RT_Audience_Reviews,US_Box_Office,Distributor,Original_Language,RT_All_Pct,IMDb_Score,Audience_Reviews,US_Users,Non_US_Users,Male_Reviews,Female_Reviews,Male_Score,Female_Score,IMDb_Score_Pct
0,17761,The Shawshank Redemption,Frank Darabont,R,Drama,142,1994,43,21,7041,91,98,82,250k+,$27.3M,Columbia Pictures,English,94.5,9.3,2666904,566032,1672983,1475251,294889,9.3,9.2,93
1,17762,The Godfather,Francis Ford Coppola,R,"Crime, Drama",175,1972,30,32,7042,97,98,149,250k+,$134.8M,Paramount Pictures,English,97.5,9.2,1848055,417566,1161734,1072921,166441,9.2,8.9,92
2,17763,The Dark Knight,Christopher Nolan,PG-13,"Action, Crime, Drama",152,2008,163,160,7043,94,94,345,250k+,$2.0M,Warner Bros.,English,94.0,9.0,2639814,621027,1646769,1490958,267216,9.0,8.7,90
3,17764,The Godfather Part II,Francis Ford Coppola,R,"Crime, Drama",202,1974,21,17,7044,96,97,123,250k+,Not Available,Paramount Pictures,English,96.5,9.0,1265636,282291,805848,759883,105690,9.0,8.7,90
4,17765,12 Angry Men,Sidney Lumet,Approved,"Crime, Drama",96,1957,13,17,7045,100,97,58,100k+,Not Available,"Criterion Collection, MGM/UA Home Entertainmen...",English,98.5,9.0,787589,187528,509017,453568,76860,9.0,8.9,90
5,17766,Schindler's List,Steven Spielberg,R,"Biography, Drama, History",195,1993,49,91,7046,98,97,130,250k+,$96.6M,Universal Pictures,English,97.5,9.0,1350290,291447,865722,758414,171042,9.0,9.0,90
6,17767,The Lord of the Rings: The Return of the King,Peter Jackson,PG-13,"Action, Adventure, Drama",201,2003,124,209,7047,93,86,274,250k+,$377.8M,New Line Cinema,English,89.5,9.0,1838370,396267,1171402,1041077,220902,9.0,8.9,90
7,17768,Pulp Fiction,Quentin Tarantino,R,"Crime, Drama",154,1994,75,70,7048,92,96,113,250k+,Not Available,Miramax Films,English,94.0,8.9,2041472,463666,1280206,1155104,222242,8.9,8.5,89
8,17769,The Lord of the Rings: The Fellowship of the Ring,Peter Jackson,PG-13,"Action, Adventure, Drama",178,2001,126,121,7049,91,95,234,250k+,$315.5M,"Warner Bros., New Line Cinema, Argentina Video...",English,93.0,8.8,1867531,406279,1182624,1038124,232568,8.8,8.8,88
9,177610,"The Good, the Bad and the Ugly",Sergio Leone,R,"Adventure, Western",178,1966,6,3,70410,97,97,75,100k+,Not Available,"Metro-Goldwyn-Mayer, United Artists",English,97.0,8.8,759951,168706,491783,495737,49542,8.9,8.4,88
